# <font color="Blue">Abstract [EDIT]</font>

In this notebook, we delve into the utilization of LangChain and its 'prompt template' function to create prompt for giving interview questions.

**<font color="red">Our goal</font>** is to craft an effective prompt tailored to providing interview questions in the context of specific jobs and companies.  

We used following knowledge:  
Prompt Engineering Skills:  
1. Placeholder;
2. Persona.


<font color="green"> Conclusion: </font>  
Through this notebook, we found that at least for Amazon and S&P Global, which are famous companies, do not need detailed information to get good interview questions.  
In my opinion,  
1. for those big companies, we can only use their [company names] and the [role title] that we are applying.  
2. maybe for some small companies, users can provide [company description] that they found might help them prepare the job interview.  
3. for any company, if users know some information, like Amazon Leadership Principles, they could mention specifically in [company description] in case the model did not pick up by themselve.

**<font color='red'> Notice: </font>**    
**<font color='red'> There are some checkpoints in this notebook. They are for saving time and resources to rerun the above code.</font>**

# Prepare the notebook for use

## Install packages & import libraries

In [1]:
# Download required packages to Colab
!pip install -q langchain
!pip install -q openai
!pip install -q chromadb
!pip install -q tiktoken
!pip install -q duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.9 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 817, in _parseNoCache
    loc, tokens = self.parseImpl(instring, pre_loc, doActions)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 3864, in parseImpl
    loc, resultlis

In [2]:
# Import required libraries
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain import ConversationChain
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import os
from getpass import getpass

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

import pandas as pd
import csv

ModuleNotFoundError: ignored

## OpenAI API key

In [ ]:
openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

## Connect Google Drive

## Create .csv files to collect info from this notebook

Since the results we got from LLMs will be different everytime, we want to use .csv files to track those results which could help use go back to see what we can do or what we did to improve the outputs.

In [ ]:
# with open('/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv', 'w', newline='') as csvfile:
#     fieldnames = ['comb_num','interviewer_role','company_name','resume','role_title','role_description','company_description']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#     writer.writeheader()
# var_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv")

In [ ]:
# with open('/content/drive/MyDrive/Capstone Project - Mock Interview App/answer_evaluation_prompt_result.csv', 'w', newline='') as csvfile:
#     fieldnames = ['model', 'question_answer_combination', 'variable_combination', 'template', 'output']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#     writer.writeheader()
#     #writer.writerow({'input': user_input, 'output': result})

In [ ]:
# with open('/content/drive/MyDrive/Capstone Project - Mock Interview App/interview_question_answer_combination.csv', 'w', newline='') as csvfile:
#     fieldnames = ['comb_num', 'var_comb_num', 'interview_question', 'question_type', 'question_answer']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#     writer.writeheader()

In [ ]:
var_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv")
prompt_result = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/answer_evaluation_prompt_result.csv")
qa_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/interview_question_answer_combination.csv")

In [ ]:
var_comb

In [ ]:
prompt_result

In [ ]:
qa_comb

## Define Parameters

### 3 interview questions

In [ ]:
var_comb_num = 1

# inter_q_1 = "Describe a challenging project you worked on. What was your role and how did you handle it?"
# inter_q_2 = "How do you handle disagreements with colleagues?"
# inter_q_3 = "How do you stay motivated and manage your time effectively?"

# q_type = "behavioral"

### Example answers for questions

#### question 1 answers

In [ ]:
q1_a1 = """
In my last role as a project manager, I led a team on a high-stakes software
development project. We faced a tight deadline and unexpected technical hurdles.
I organized extra brainstorming sessions, delegated tasks based on individual
strengths, and maintained open communication with stakeholders. We delivered the
project on time, and it was well-received by the client.
""" # 9/10

q1_a2 = """
I was part of a marketing campaign that faced budget cuts. My role was to design
the digital ads. I had to be creative with limited resources, so I focused on
leveraging free online tools and optimizing our existing content. The campaign
was successful, though it didn't reach all our targets.
""" # 7/10

q1_a3 = """
As a junior analyst, I worked on a data migration project that was complex. My
role was to assist senior analysts. I ensured that I was thorough in my tasks
and asked for feedback regularly. The project was successful, but I wish I had
a more active role.
""" # 6/10

q1_a4 = """
I worked on a challenging project last year, but honestly, I didn't have much of
a role. I just did what I was told and didn't really understand the bigger picture.
""" # 3/10

q1_a5 = """
In my previous job, there was a project that was really tough. I found it
overwhelming, so I mostly kept to the tasks I was comfortable with and didn't
venture much beyond that.
""" # 2/10

q1_a6 = """
I was involved in a project that didn't go well. Our team had many issues, and
I think it was mostly due to poor management. I just tried to stay out of the
way and do my part.
""" # 3/10

q1_a7 = """
I led a cross-functional team on a critical product launch that faced major
hurdles due to supply chain disruptions. As the project manager, I quickly
re-strategized, identifying alternative suppliers and adjusting our timelines
while maintaining open communication with all stakeholders. My proactive approach
and contingency planning enabled us to launch successfully, albeit with a slight
delay, but with no compromise on quality.
""" # 10/10

q1_a8 = """
In my role as an IT specialist, I was tasked with leading a critical data migration
project under a tight deadline. Recognizing the complexity, I implemented a
phased approach, conducted thorough risk assessments, and held regular team meetings
to ensure everyone was aligned and on track. My emphasis on meticulous planning
and team coordination led to a successful migration with zero data loss and
minimal downtime.
""" # 10/10

#### question 2 answers

In [ ]:
q2_a1 = """
I believe in open communication and empathy. In a previous job, a colleague and
I disagreed on a sales strategy. I initiated a meeting to discuss our viewpoints.
We combined our ideas and developed a hybrid strategy that proved effective.
""" # 8/10

q2_a2 = """
Disagreements can be challenging. I usually try to avoid confrontation and go
with the majority's opinion to keep the peace.
""" # 4/10

q2_a3 = """
In my last role, I disagreed with a teammate over a design choice. We presented
our ideas to the team, and I accepted the team's decision even though it wasn't
my preference. I learned a lot from the process.
""" # 7/10

q2_a4 = """
I don't like conflicts, so I usually just keep my opinions to myself. It's easier
to just agree with others even if I don't think it's the best idea.
""" # 2/10

q2_a5 = """
Disagreements are tough. In the past, I've had arguments with colleagues. I tend
to stick to my point because I believe in my ideas strongly.
""" # 3/10

q2_a6 = """
I usually talk to other colleagues about the disagreement and try to get them on
my side. I find it's better to have support when there's a conflict.
""" # 2/10

q2_a7 = """
I believe in addressing disagreements constructively. In one instance, a colleague
and I had differing views on a marketing strategy. I initiated a meeting to discuss
our perspectives in-depth. Through active listening and collaborative discussion,
we combined the best elements of our ideas, resulting in a more robust strategy
that increased our campaign's success by 30%.
""" # 10/10

q2_a8 = """
I approach disagreements with openness and a focus on problem-solving. Once, when
a project proposal I was working on with a colleague led to a difference in opinion,
we decided to seek input from other team members and conducted a small workshop.
This approach not only resolved our disagreement but also brought in valuable new
insights, leading to a stronger, team-endorsed proposal.
""" # 10/10

#### question 3 answers

In [ ]:
q3_a1 = """
I stay motivated by setting clear, achievable goals. For time management, I use
the Pomodoro Technique, breaking my work into focused intervals with short breaks.
This keeps me productive and prevents burnout.
""" # 8/10

q3_a2 = """
Motivation can be tough, especially with routine tasks. I try to keep a big-picture
perspective and remind myself how my work contributes to the company's goals. As
for time management, I prioritize tasks each morning and adjust as needed
throughout the day.
""" # 7/10

q3_a3 = """
I find motivation in challenges and learning new things. I manage my time by
setting weekly goals and reviewing my progress every Friday. This helps me stay
on track and adjust my plans for the following week.
""" # 8/10

q3_a4 = """
I struggle with motivation, especially on long projects. I don't really have a
time management strategy, I just do things as they come up.
""" # 2/10

q3_a5 = """
I find it hard to stay motivated. I often procrastinate and then have to rush to
meet deadlines. It's stressful, but that's just how I've always worked.
""" # 1/10

q3_a6 = """
I tend to work in bursts. Sometimes I'm really motivated, other times not so
much. I don't really plan my time; I just work more when the deadlines are near.
""" # 3/10

q3_a7 = """
My motivation comes from setting challenging yet achievable goals. I use a
combination of the Eisenhower Matrix for prioritizing tasks and the Pomodoro
Technique for focused work sessions. This approach helped me successfully manage
multiple projects simultaneously, leading to a 20% increase in efficiency in my
department.
""" # 10/10

q3_a8 = """
To stay motivated, I align my work with my personal and career growth goals. For
time management, I meticulously plan my weeks every Sunday, setting clear
objectives for each day, and review my progress nightly. This method was instrumental
in my recent project, where I managed to deliver high-quality work two weeks ahead
of schedule.
""" # 10/10

### Store info

In [ ]:
# qa_comb

In [ ]:
# new_qa_comb = pd.DataFrame({
#     'comb_num': list(range(1, 3*8+1)),
#     'var_comb_num': [1]*(3*8),
#     'interview_question': [inter_q_1]*8 + [inter_q_2]*8 + [inter_q_3]*8,
#     'question_type': [q_type]*(3*8),
#     'question_answer': [q1_a1, q1_a2, q1_a3, q1_a4, q1_a5, q1_a6, q1_a7, q1_a8,
#                         q2_a1, q2_a2, q2_a3, q2_a4, q2_a5, q2_a6, q2_a7, q2_a8,
#                         q3_a1, q3_a2, q3_a3, q3_a4, q3_a5, q3_a6, q3_a7, q3_a8]
# })

# qa_comb = pd.concat([qa_comb, new_qa_comb], ignore_index=True)

In [ ]:
qa_comb

# Model selection

In [10]:
chat = ChatOpenAI(model_name = "gpt-4")

In [11]:
# record model as ex_model
ex_model = "gpt_4"

# Answer evaluation prompts

# -> Checkpoint Start [2023.11.13]
[To avoid rerun the above code before Checkpoint End, the code between this Checkpoint Start to Checkpoint End does not need to be rerun. This will save us time and resources.]

## Prompt 1

### Extra info

I explore some evaluation matrixes for bahavioral questions. Below I asked ChatGPT 4:  
1. Give me the list of evaluation matrixes & rationales;   
2. Interviewees example answers and score with rationales;
3. Example answers with scores lower than 5/10;
4. Example answers with scores 10/10.






The following are the conversation between GPT and me:

<font color='Green'>ME:</font>  
As an experienced interviewer, what are some important aspects for you to evaluate interviewees? 1) list those aspects and rationales, 2) also give me 3 examples of interview questions and example answers from 3 different interviewees, give them a score and feedbacks

<font color='Blue'>GPT:</font>   
As an experienced interviewer, there are several key aspects that I would evaluate in interviewees. These aspects are important because they help in determining not only the candidate's skills and qualifications but also their fit within the team and organization. Here's a list of these aspects along with their rationales:

1. **Technical Skills**: This is about understanding the candidate's ability to perform the specific tasks required for the job. For roles that require specific technical knowledge or skills, it's crucial to assess the candidate's proficiency.

2. **Problem-Solving Abilities**: This evaluates how the candidate approaches and solves problems. It's important because it shows how they will handle real-world challenges in the job.

3. **Communication Skills**: Good communication is key in almost every job. This aspect assesses how well the candidate can express their ideas and understand others.

4. **Cultural Fit**: This is about assessing whether the candidate's values, behavior, and personality align with the company's culture. It's important for ensuring a harmonious work environment.

5. **Adaptability**: The ability to adapt to changing environments and demands is crucial in today’s fast-paced work settings. This shows how well the candidate can handle change and uncertainty.

6. **Teamwork and Collaboration**: This assesses the candidate’s ability to work effectively with others. Teamwork is essential for most jobs, and good collaboration skills can significantly enhance work efficiency.

7. **Leadership Qualities (if applicable)**: For roles that require leadership, it’s important to evaluate the candidate’s ability to lead, motivate, and guide others.

Now, let's move on to the three example interview questions, along with example answers from three different interviewees, their scores, and feedback:

**Question 1:** Describe a challenging project you worked on. What was your role and how did you handle it?

- **Interviewee A:** Explained a complex project, detailed their role as a team leader, and outlined the strategies they used to overcome challenges. Their answer showed strong problem-solving and leadership skills.
  - **Score:** 9/10
  - **Feedback:** Excellent answer, showcasing leadership and problem-solving skills. Could have elaborated more on teamwork aspects.

- **Interviewee B:** Gave a vague description of the project and focused more on the challenges than their role in solving them.
  - **Score:** 5/10
  - **Feedback:** The response needs more detail about your specific contributions and problem-solving strategies.

- **Interviewee C:** Discussed a project with enthusiasm, highlighted their role in detail, but lacked clarity on the outcome.
  - **Score:** 7/10
  - **Feedback:** Good energy and clear role definition, but focus on explaining the results and your direct impact on them.

**Question 2:** How do you handle disagreements with colleagues?

- **Interviewee A:** Provided a balanced view, emphasizing respectful communication and seeking common ground.
  - **Score:** 8/10
  - **Feedback:** Strong communication skills shown, consider adding more about conflict resolution techniques.

- **Interviewee B:** Answered with an example where they avoided conflict, which led to unresolved issues.
  - **Score:** 4/10
  - **Feedback:** It's important to address conflicts directly. Consider discussing how you would engage constructively in future situations.

- **Interviewee C:** Discussed a specific incident, their approach to understanding the other side, and how they achieved a compromise.
  - **Score:** 9/10
  - **Feedback:** Excellent approach to conflict resolution, demonstrating empathy and effective communication.

**Question 3:** How do you stay motivated and manage your time effectively?

- **Interviewee A:** Talked about setting personal goals and using productivity tools, but lacked depth in their response.
  - **Score:** 6/10
  - **Feedback:** Good use of tools and goal-setting. Elaborate more on how these methods impact your daily work.

- **Interviewee B:** Shared a detailed routine involving prioritization, breaks, and regular reviews of progress.
  - **Score:** 8/10
  - **Feedback:** Great detail on time management strategies. Include how you adapt these methods during particularly busy or stressful periods.

- **Interviewee C:** Mentioned only intrinsic motivation without specific time management techniques.
  - **Score:** 5/10
  - **Feedback:** While intrinsic motivation is important, discuss specific strategies you use for managing time effectively.

<font color='Green'>ME:</font>  
Could you regenerate those examples, scores, and rationales? This time, instead of giving what interviewees did, generate real answers for those interview questions. Try it again. You can do it!

<font color='Blue'>GPT:</font>  
Absolutely! Let's create more realistic examples of interview answers, complete with scores and feedback for each.

**Question 1**: Describe a challenging project you worked on. What was your role and how did you handle it?

- **Interviewee A's Answer:** "In my last role as a project manager, I led a team on a high-stakes software development project. We faced a tight deadline and unexpected technical hurdles. I organized extra brainstorming sessions, delegated tasks based on individual strengths, and maintained open communication with stakeholders. We delivered the project on time, and it was well-received by the client."
  - **Score:** 9/10
  - **Feedback:** Excellent response demonstrating strong leadership and problem-solving skills. Great example of effective communication and team management under pressure.

- **Interviewee B's Answer:** "I was part of a marketing campaign that faced budget cuts. My role was to design the digital ads. I had to be creative with limited resources, so I focused on leveraging free online tools and optimizing our existing content. The campaign was successful, though it didn't reach all our targets."
  - **Score:** 7/10
  - **Feedback:** Good demonstration of adaptability and creativity under constraints. Could improve by discussing how you measured the campaign's success and learned from the experience.

- **Interviewee C's Answer:** "As a junior analyst, I worked on a data migration project that was complex. My role was to assist senior analysts. I ensured that I was thorough in my tasks and asked for feedback regularly. The project was successful, but I wish I had a more active role."
  - **Score:** 6/10
  - **Feedback:** Good job on showing dedication and a willingness to learn. For future responses, focus more on your direct contributions and any proactive steps you took.

**Question 2**: How do you handle disagreements with colleagues?

- **Interviewee A's Answer:** "I believe in open communication and empathy. In a previous job, a colleague and I disagreed on a sales strategy. I initiated a meeting to discuss our viewpoints. We combined our ideas and developed a hybrid strategy that proved effective."
  - **Score:** 8/10
  - **Feedback:** Strong approach to conflict resolution, showcasing your ability to listen and collaborate. Great example of turning disagreement into a positive outcome.

- **Interviewee B's Answer:** "Disagreements can be challenging. I usually try to avoid confrontation and go with the majority's opinion to keep the peace."
  - **Score:** 4/10
  - **Feedback:** It's important to address conflicts constructively. Consider discussing how you could actively engage in finding a solution rather than avoiding confrontation.

- **Interviewee C's Answer:** "In my last role, I disagreed with a teammate over a design choice. We presented our ideas to the team, and I accepted the team's decision even though it wasn't my preference. I learned a lot from the process."
  - **Score:** 7/10
  - **Feedback:** Good example of teamwork and respecting collective decisions. You might strengthen your answer by sharing how you contributed to the final solution or what you learned from the experience.

**Question 3**: How do you stay motivated and manage your time effectively?

- **Interviewee A's Answer:** "I stay motivated by setting clear, achievable goals. For time management, I use the Pomodoro Technique, breaking my work into focused intervals with short breaks. This keeps me productive and prevents burnout."
  - **Score:** 8/10
  - **Feedback:** Effective demonstration of personal motivation and time management strategies. Could enhance your answer by sharing a specific instance where these techniques helped you overcome a challenge.

- **Interviewee B's Answer:** "Motivation can be tough, especially with routine tasks. I try to keep a big-picture perspective and remind myself how my work contributes to the company's goals. As for time management, I prioritize tasks each morning and adjust as needed throughout the day."
  - **Score:** 7/10
  - **Feedback:** Good insights into maintaining motivation. Your time management approach shows adaptability. Consider discussing how you handle unexpected high-priority tasks.

- **Interviewee C's Answer:** "I find motivation in challenges and learning new things. I manage my time by setting weekly goals and reviewing my progress every Friday. This helps me stay on track and adjust my plans for the following week."
  - **Score:** 8/10
  - **Feedback:** Strong focus on continuous learning and goal-oriented time management. It would be beneficial to include how you balance long-term and short-term tasks.

<font color='Green'>ME:</font>  
Great job! Do you mind generating 3 more examples for each interview question above? This time, please generate answers that have score lower than 5/10.

<font color='Blue'>GPT:</font>  
Certainly! Here are additional examples of interview answers for each question, this time with scores below 5/10, indicating areas for improvement.

**Question 1**: Describe a challenging project you worked on. What was your role and how did you handle it?

- **Interviewee D's Answer:** "I worked on a challenging project last year, but honestly, I didn't have much of a role. I just did what I was told and didn't really understand the bigger picture."
  - **Score:** 3/10
  - **Feedback:** The response lacks detail and initiative. It's important to show your active participation and understanding of your role in a project.

- **Interviewee E's Answer:** "In my previous job, there was a project that was really tough. I found it overwhelming, so I mostly kept to the tasks I was comfortable with and didn't venture much beyond that."
  - **Score:** 2/10
  - **Feedback:** This answer shows a reluctance to engage with challenges. Consider discussing how you could approach difficulties more proactively in the future.

- **Interviewee F's Answer:** "I was involved in a project that didn't go well. Our team had many issues, and I think it was mostly due to poor management. I just tried to stay out of the way and do my part."
  - **Score:** 3/10
  - **Feedback:** The response shows a lack of engagement and responsibility. Reflect on how you could contribute positively in difficult situations.

**Question 2**: How do you handle disagreements with colleagues?

- **Interviewee D's Answer:** "I don't like conflicts, so I usually just keep my opinions to myself. It's easier to just agree with others even if I don't think it's the best idea."
  - **Score:** 2/10
  - **Feedback:** Avoiding conflict can lead to unresolved issues. It's important to learn how to express your views constructively and engage in healthy discussions.

- **Interviewee E's Answer:** "Disagreements are tough. In the past, I've had arguments with colleagues. I tend to stick to my point because I believe in my ideas strongly."
  - **Score:** 3/10
  - **Feedback:** This approach can create tension. Focus on developing skills for collaborative problem-solving and understanding different perspectives.

- **Interviewee F's Answer:** "I usually talk to other colleagues about the disagreement and try to get them on my side. I find it's better to have support when there's a conflict."
  - **Score:** 2/10
  - **Feedback:** This strategy can lead to workplace division. Consider direct and open communication with the colleague you disagree with instead.

**Question 3**: How do you stay motivated and manage your time effectively?

- **Interviewee D's Answer:** "I struggle with motivation, especially on long projects. I don't really have a time management strategy, I just do things as they come up."
  - **Score:** 2/10
  - **Feedback:** Consider developing a more structured approach to your work and finding personal motivation drivers to improve productivity.

- **Interviewee E's Answer:** "I find it hard to stay motivated. I often procrastinate and then have to rush to meet deadlines. It's stressful, but that's just how I've always worked."
  - **Score:** 1/10
  - **Feedback:** This approach is unsustainable and can lead to burnout. Look into time management techniques and strategies to tackle procrastination.

- **Interviewee F's Answer:** "I tend to work in bursts. Sometimes I'm really motivated, other times not so much. I don't really plan my time; I just work more when the deadlines are near."
  - **Score:** 3/10
  - **Feedback:** This inconsistent approach can be risky. Developing a more steady and planned approach to work can help with both motivation and time management.

<font color='Green'>ME:</font>  
Wonderful! Please generate 2 10/10 answers for each interview question above.

<font color='Blue'>GPT:</font>  
Certainly! Here are examples of 10/10 answers for each interview question, demonstrating exemplary responses.

**Question 1**: Describe a challenging project you worked on. What was your role and how did you handle it?

- **Interviewee G's Answer:** "I led a cross-functional team on a critical product launch that faced major hurdles due to supply chain disruptions. As the project manager, I quickly re-strategized, identifying alternative suppliers and adjusting our timelines while maintaining open communication with all stakeholders. My proactive approach and contingency planning enabled us to launch successfully, albeit with a slight delay, but with no compromise on quality."
  - **Score:** 10/10
  - **Feedback:** This answer excellently showcases leadership, problem-solving, adaptability, and effective communication. It demonstrates a clear understanding of how to manage complex projects under pressure.

- **Interviewee H's Answer:** "In my role as an IT specialist, I was tasked with leading a critical data migration project under a tight deadline. Recognizing the complexity, I implemented a phased approach, conducted thorough risk assessments, and held regular team meetings to ensure everyone was aligned and on track. My emphasis on meticulous planning and team coordination led to a successful migration with zero data loss and minimal downtime."
  - **Score:** 10/10
  - **Feedback:** Outstanding response highlighting technical expertise, strategic planning, and team management skills. Your focus on risk mitigation and detailed execution is commendable.

**Question 2**: How do you handle disagreements with colleagues?

- **Interviewee G's Answer:** "I believe in addressing disagreements constructively. In one instance, a colleague and I had differing views on a marketing strategy. I initiated a meeting to discuss our perspectives in-depth. Through active listening and collaborative discussion, we combined the best elements of our ideas, resulting in a more robust strategy that increased our campaign's success by 30%."
  - **Score:** 10/10
  - **Feedback:** Excellent approach to conflict resolution. Your ability to listen, understand different viewpoints, and create a unified solution is exemplary.

- **Interviewee H's Answer:** "I approach disagreements with openness and a focus on problem-solving. Once, when a project proposal I was working on with a colleague led to a difference in opinion, we decided to seek input from other team members and conducted a small workshop. This approach not only resolved our disagreement but also brought in valuable new insights, leading to a stronger, team-endorsed proposal."
  - **Score:** 10/10
  - **Feedback:** This answer demonstrates a mature approach to resolving conflicts. Involving the team and turning a disagreement into an opportunity for broader engagement and idea generation is commendable.

**Question 3**: How do you stay motivated and manage your time effectively?

- **Interviewee G's Answer:** "My motivation comes from setting challenging yet achievable goals. I use a combination of the Eisenhower Matrix for prioritizing tasks and the Pomodoro Technique for focused work sessions. This approach helped me successfully manage multiple projects simultaneously, leading to a 20% increase in efficiency in my department."
  - **Score:** 10/10
  - **Feedback:** This response shows a sophisticated understanding of time management and personal motivation. The mention of specific techniques and their impact on your work is impressive.

- **Interviewee H's Answer:** "To stay motivated, I align my work with my personal and career growth goals. For time management, I meticulously plan my weeks every Sunday, setting clear objectives for each day, and review my progress nightly. This method was instrumental in my recent project, where I managed to deliver high-quality work two weeks ahead of schedule."
  - **Score:** 10/10
  - **Feedback:** Excellent strategy for maintaining motivation and managing time. Your structured approach and dedication to continuous improvement are exemplary.


**HERE** <font color='Blue'> are evaluation matrixes:</font>  

<font color='green'> Interview Evaluation Matrixes: </font>   
1. **Technical Skills**: This is about understanding the candidate's ability to perform the specific tasks required for the job. For roles that require specific technical knowledge or skills, it's crucial to assess the candidate's proficiency.

2. **Problem-Solving Abilities**: This evaluates how the candidate approaches and solves problems. It's important because it shows how they will handle real-world challenges in the job.

3. **Communication Skills**: Good communication is key in almost every job. This aspect assesses how well the candidate can express their ideas and understand others.

4. **Cultural Fit**: This is about assessing whether the candidate's values, behavior, and personality align with the company's culture. It's important for ensuring a harmonious work environment.

5. **Adaptability**: The ability to adapt to changing environments and demands is crucial in today’s fast-paced work settings. This shows how well the candidate can handle change and uncertainty.

6. **Teamwork and Collaboration**: This assesses the candidate’s ability to work effectively with others. Teamwork is essential for most jobs, and good collaboration skills can significantly enhance work efficiency.

7. **Leadership Qualities (if applicable)**: For roles that require leadership, it’s important to evaluate the candidate’s ability to lead, motivate, and guide others.


**HERE** <font color='Blue'> are 3 examples of behavioral interview questions:</font>  

<font color='green'>Question 1: </font>   
Describe a challenging project you worked on. What was your role and how did you handle it?

<font color='green'>Question 2: </font>   
How do you handle disagreements with colleagues?

<font color='green'>Question 3: </font>   
How do you stay motivated and manage your time effectively?

### Code

In [15]:
# set up input variables for prompt
input_var_1 = ["interview_questions", "question_answers", "role_title",
               "role_description", "company_name", "company_description"]

In [16]:
template_1 = """
You are an expert evaluator of interview responses. Given an interview questions: {interview_questions}
 and a corresponding answer: {question_answers}, your job is to evaluate each answer based
 on the following seven key aspects: relevance to question, technical skills,
 problem-solving abilities, communication skills, cultural fit, adaptability,
 teamwork and collaboration, and leadership qualities (if applicable).

Relevance to Question: "Does the answer directly address the question asked?"
High Relevance: The answer directly and comprehensively addresses the question.
Low Relevance: The answer is off-topic, vague, or fails to address the question.

Technical Skills: This is about understanding the candidate's ability to perform
 the specific tasks required for the job: {role_title} and job description: {role_description}. For roles that require specific
 technical knowledge or skills, it's crucial to assess the candidate's proficiency.
 "Does the answer demonstrate the necessary technical skills or knowledge for the job?"
High Competency: The answer shows in-depth understanding and expertise.
Low Competency: The answer reveals a lack of necessary technical knowledge.

Problem-Solving Abilities: This evaluates how the candidate approaches and solves
 problems. It's important because it shows how they will handle real-world
 challenges in the job. "Does the answer exhibit the candidate's ability to solve
 problems?"
Strong Problem-Solving: The answer showcases effective and creative problem-solving strategies.
Weak Problem-Solving: The answer lacks evidence of problem-solving abilities.

Communication Skills: Good communication is key in almost every job. This aspect
 assesses how well the candidate can express their ideas and understand others.
 "How well does the candidate articulate their thoughts?"
Strong Communication: The answer is clear, concise, and well-structured.
Poor Communication: The answer is confusing, overly verbose, or poorly structured.

Cultural Fit: This is about assessing whether the candidate's values, behavior,
and personality align with the {company_name}'s company culture. Here is the company
description you can reference "{company_description}".
It's important for ensuring a harmonious work environment. "Does the answer align
with the {company_name}'s company values and culture?"
Good Fit: The answer reflects alignment with the company's values and culture.
Poor Fit: The answer indicates potential conflicts with the company's culture.

Adaptability: The ability to adapt to changing environments and demands is crucial
 in today’s fast-paced work settings. This shows how well the candidate can
 handle change and uncertainty. "Does the answer demonstrate the candidate's
 ability to adapt to changes?"
High Adaptability: The answer shows flexibility and a positive approach to change.
Low Adaptability: The answer suggests rigidity or discomfort with change.

Teamwork and Collaboration: This assesses the candidate’s ability to work
effectively with others. Teamwork is essential for most jobs, and good collaboration
 skills can significantly enhance work efficiency.
Leadership Qualities (if applicable): For roles that require leadership, it’s
important to evaluate the candidate’s ability to lead, motivate, and guide others.
"Does the answer reflect the candidate's ability to work in a team or lead effectively?"
Strong Teamwork/Leadership: The answer highlights effective collaboration or leadership qualities.
Weak Teamwork/Leadership: The answer shows a lack of teamwork skills or ineffective leadership.


For each interview question and answer provide at the beginning, rate the response based on the above aspects.
Provide a score and feedback for each aspect in 10/10 scale.
Offer an overall evaluation of the answer's strengths and areas for improvement.
"""

In [17]:
prompt_11 = PromptTemplate(
    input_variables = input_var_1,
    template = template_1
)

In [27]:
var_comb_num =

NameError: ignored

In [21]:
job_title = var_comb.role_title[0]
job_des = var_comb.role_description[0]
comp_name = var_comb.company_name[0]
comp_des = var_comb.company_description[0]

#### Q1 A1 [9.3/10]

In [34]:
# set up prameters
qa_comb_num = 1
inter_q = qa_comb.interview_question[0]
q_a = qa_comb.question_answer[0]

In [36]:
# record those parameters as ex_var_comb
ex_var_comb = 1 # this means we use the 1st combination of information in var_comb.csv dataset; this helps us tack information

In [38]:
#from pydantic.typing import resolve_annotations

new_prompt_11 = prompt_11.format(interview_questions = inter_q,
                                 question_answers = q_a,
                                 role_title = job_title,
                                 role_description = job_des,
                                 company_name = comp_name,
                                 company_description = comp_des)
result_11 = chat([HumanMessage(content=new_prompt_11)])

In [40]:
print(result_11.content)

Relevance to Question: 
Score: 10/10 
Feedback: The candidate clearly describes a challenging project that they worked on, outlining their role and the strategies they used to handle the situation.

Technical Skills: 
Score: 7/10 
Feedback: The candidate shows good project management skills, but specific technical skills related to AI/ML software development were not clearly mentioned.

Problem-Solving Abilities: 
Score: 9/10 
Feedback: The candidate demonstrates strong problem-solving abilities by organizing extra brainstorming sessions, delegating tasks, and maintaining open communication with stakeholders to meet the project deadline.

Communication Skills: 
Score: 10/10 
Feedback: The candidate effectively communicates their thoughts, outlining the problem and their strategies in a clear, concise manner.

Cultural Fit: 
Score: 8/10 
Feedback: The candidate shows good project management skills and a strong ability to handle pressure, which aligns with S&P Global's fast-paced and dat

##### store output

In [43]:
# ex_model, qa_comb_num, var_comb_num, template_1, result_11.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_1],
    'output': [result_11.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [44]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...


#### Q1 A2 [7.3/10]

In [50]:
# set up prameters
qa_comb_num = qa_comb.comb_num[1]

q_a = qa_comb.question_answer[1]

In [47]:
prompt_11 = PromptTemplate(
    input_variables = input_var_1,
    template = template_1
)

new_prompt_12 = prompt_11.format(interview_questions = inter_q,
                                 question_answers = q_a,
                                 role_title = job_title,
                                 role_description = job_des,
                                 company_name = comp_name,
                                 company_description = comp_des)
result_12 = chat([HumanMessage(content=new_prompt_12)])

In [48]:
print(result_12.content)

Relevance to Question: 10/10
The candidate's response directly addresses the question, clearly detailing a challenging project they worked on and their role in it.

Technical Skills: 3/10
The candidate discusses their role in a marketing campaign, but does not mention any technical skills related to software development, AI/ML, or the other technical requirements listed in the job description. 

Problem-Solving Abilities: 8/10
The candidate shows good problem-solving abilities. They faced a challenge due to budget cuts and found a creative solution by leveraging free online tools and optimizing existing content.

Communication Skills: 8/10
The candidate communicates their ideas clearly and concisely, explaining the challenge they faced and how they handled it.

Cultural Fit: 7/10
The candidate shows a commitment to achieving targets even in challenging circumstances, which aligns with S&P Global's focus on providing essential intelligence and helping clients make informed decisions. Ho

##### store output

In [51]:
# ex_model, qa_comb_num, var_comb_num, template_1, result_12.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_1],
    'output': [result_12.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [52]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...


#### Q1 A3 [6.6/10]

In [53]:
# set up prameters
answer_num = 3
ind = answer_num-1

qa_comb_num = qa_comb.comb_num[ind]

q_a = qa_comb.question_answer[ind]

In [54]:
new_prompt_13 = prompt_11.format(interview_questions = inter_q,
                                 question_answers = q_a,
                                 role_title = job_title,
                                 role_description = job_des,
                                 company_name = comp_name,
                                 company_description = comp_des)
result_13 = chat([HumanMessage(content=new_prompt_13)])

In [55]:
print(result_13.content)

Relevance to Question: 9/10
The candidate directly addresses the question by describing a specific project they worked on and their role in it. However, they could have expanded more on the challenges they faced.

Technical Skills: 5/10
The candidate mentions working on a data migration project, which shows some technical competency. However, they do not mention any specific technical skills or tools used, such as AI or ML as required in the job description.

Problem-Solving Abilities: 5/10
The candidate doesn't clearly specify any problems they encountered or how they solved them. They only mention wishing they had a more active role, which doesn't provide insight into their problem-solving abilities.

Communication Skills: 8/10
The candidate communicates their thoughts in a clear and concise manner. However, they could have expanded more on their experience and the challenges they faced.

Cultural Fit: 6/10
The candidate demonstrates a willingness to learn and receive feedback, align

##### store output

In [56]:
# ex_model, qa_comb_num, var_comb_num, template_1, result_13.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_1],
    'output': [result_13.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [57]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...


#### [checkpoint] output data

In [60]:
#prompt_result
prompt_result.to_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/answer_evaluation_prompt_result.csv", index=False)

In [61]:
#qa_comb
qa_comb.to_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/interview_question_answer_combination.csv", index=False)

# -> Checkpoint End [2023.11.13]
[To avoid rerun the previous code, we will output above info into .csv files that we create. So next time when we come back, we can just start from this checkpoint.]

## Prompt 1

### Code

#### Q1 A5 [4/10]

In [1]:
# set up prameters
answer_num = 5
ind = answer_num-1

qa_comb_num = qa_comb.comb_num[ind]

inter_q = qa_comb.interview_question[ind]
q_a = qa_comb.question_answer[ind]

NameError: ignored

In [2]:
inter_q

NameError: ignored

In [ ]:
q_a

In [ ]:
new_prompt = prompt_11.format(interview_questions = inter_q,
                                 question_answers = q_a,
                                 role_title = job_title,
                                 role_description = job_des,
                                 company_name = comp_name,
                                 company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [ ]:
print(result.content)

##### store output

In [ ]:
# ex_model, qa_comb_num, var_comb_num, template_1, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_1],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [ ]:
prompt_result

#### Q2 A4 [5.2/10]

In [ ]:
# set up prameters
question_num = 2
answer_num = 4
ind = (question_num-1) * 8 + answer_num - 1

qa_comb_num = qa_comb.comb_num[ind]

inter_q = qa_comb.interview_question[ind]
q_a = qa_comb.question_answer[ind]

In [ ]:
inter_q

In [ ]:
q_a

In [ ]:
new_prompt = prompt_11.format(interview_questions = inter_q,
                                 question_answers = q_a,
                                 role_title = job_title,
                                 role_description = job_des,
                                 company_name = comp_name,
                                 company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [ ]:
print(result.content)

##### store output

In [ ]:
# ex_model, qa_comb_num, var_comb_num, template_1, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_1],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [ ]:
prompt_result

#### Q3 A7 [7.3/10]

In [ ]:
# set up prameters
question_num = 3
answer_num = 7
ind = (question_num-1) * 8 + answer_num - 1

qa_comb_num = qa_comb.comb_num[ind]

inter_q = qa_comb.interview_question[ind]
q_a = qa_comb.question_answer[ind]

In [ ]:
inter_q

In [ ]:
q_a

In [ ]:
new_prompt = prompt_11.format(interview_questions = inter_q,
                                 question_answers = q_a,
                                 role_title = job_title,
                                 role_description = job_des,
                                 company_name = comp_name,
                                 company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [ ]:
print(result.content)

##### store output

In [ ]:
# ex_model, qa_comb_num, var_comb_num, template_1, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_1],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [ ]:
prompt_result

#### output data

In [ ]:
#prompt_result
prompt_result.to_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/answer_evaluation_prompt_result.csv", index=False)

# -> Checkpoint End [2023.11.14 12:17PM]

# **Continue at 1_5_2_answer_evaluation_prompt_behavioral_question.ipynb**